# Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 65)
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, RepeatedKFold, cross_val_score
from sklearn.metrics import accuracy_score, log_loss
from joblib import dump, load
import warnings
import json
warnings.filterwarnings('ignore')
from pandarallel import pandarallel
# pandarallel.initialize(shm_size_mb=32000, nb_workers=24)
%load_ext autotime

# Read in data and prepare target variable

In [2]:
df = pd.read_csv("usa_small.csv")

time: 10.7 s


In [3]:
df = df[df['MARST'].isin([1,3,4])]

df['MARST'] = df['MARST'].replace(1,0)
df['MARST'] = df['MARST'].replace([3,4],1)

time: 1.91 s


# Filter out entries with revealing information

In [4]:
df = df[(df["FERTYR"]!=8)]
df = df[(df["NCOUPLES"]!=0)]
df = df[(df["FAMSIZE"]!=1)]
df = df[~df["MARRNO_SP"].isna()]
df = df[df["MARRNO_SP"] != 0]
df = df[df["EMPSTAT"] != 0]
df = df[~df["OCCUPATION_CATEGORY_SP"].isna()]

time: 2.89 s


# Resample to keep same distribution

In [5]:
divorce_rate = 0.5
multiplier = (1 - divorce_rate)/divorce_rate
num_married_sample = int(df["MARST"].value_counts()[1]*multiplier)

married = df[df["MARST"]==0].sample(n=num_married_sample, random_state=13)

df = df[df["MARST"]==1].append(married)

time: 523 ms


# Create training data

In [6]:
df["MARRNO"] -= 1
df["MARRNO_SP"] -= 1

time: 24.1 ms


In [7]:
X_columns = ["AGE", "AGE_SP", "MARRNO", "MARRNO_SP", "EMPSTAT", "EMPSTAT_SP", "INCTOT", "INCTOT_SP", "RACE", "RACE_SP", 
             "OCCUPATION_CATEGORY", "OCCUPATION_CATEGORY_SP", "STATEFIP"]
X = df[X_columns]

X["Age_Difference"] = abs(X["AGE"] - X["AGE_SP"])
X["Income_Difference"] = abs(X["INCTOT"] - X["INCTOT_SP"])
y = df[["MARST"]]
pd.DataFrame(columns=X_columns).to_csv("model_input.csv", index=False)

time: 102 ms


# Train and evaluate model

In [8]:
model = GradientBoostingClassifier(learning_rate=.1, n_estimators=200, max_depth=5, subsample=1.0, min_samples_split=10, random_state=33)
# model = LogisticRegression()
cv_results = cross_validate(model, X, y, cv=RepeatedKFold(n_splits=5, n_repeats=3, random_state=7), n_jobs=-1, return_train_score=True, 
                            scoring=['accuracy', 'neg_log_loss'])

time: 53.7 s


In [9]:
# model = GradientBoostingClassifier(learning_rate=.1, n_estimators=200, max_depth=5, subsample=1.0, min_samples_split=10, random_state=33)
print("Accuracy: {}".format(cv_results['test_accuracy'].mean()))
print("Log Loss: {}".format(-cv_results['test_neg_log_loss'].mean()))

Accuracy: 0.6984211194314
Log Loss: 0.5740558237424364
time: 4.61 ms


In [10]:
### Naive Model
print("Naive Accuracy: {}".format(1-y.mean()))
print("Naive Log Loss: {}".format(log_loss(y, [y.mean()]*len(y))))

Naive Accuracy: MARST    0.5
dtype: float64
Naive Log Loss: 0.6931471805599453
time: 2.22 s


# Train and save model

In [11]:
model = GradientBoostingClassifier(learning_rate=.1, n_estimators=200, max_depth=5, subsample=1.0, min_samples_split=10, random_state=33)
model.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='auto',
                           random_state=33, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

time: 17.1 s


In [12]:
dump(model, "GBC.joblib")

['GBC.joblib']

time: 51.1 ms


# Load model and make predictions

In [14]:
model = load("GBC.joblib")

time: 45.2 ms


In [15]:
pred_data = pd.read_excel("h or s model.xlsx")

time: 33.3 ms


In [53]:
pred_data["Age_Difference"] = abs(pred_data["AGE"] - pred_data["AGE_SP"])
pred_data["Income_Difference"] = abs(pred_data["INCTOT"] - pred_data["INCTOT_SP"])

time: 6.16 ms


In [54]:
pred_data['divorce_probability'] = model.predict_proba(pred_data)[:,1]

time: 10.4 ms


In [23]:
pred_data.to_csv("prediction_output.csv", index=False)

time: 13.8 ms


# Initial setup

In [2]:
df = pd.read_csv("usa_small.csv")
df = df.reindex(columns=(['MARST'] + list([a for a in df.columns if a != 'MARST']) ))

time: 11.1 s


In [3]:
with open("variable_codes/occupation_ranges.txt", "r") as f:
    occupation_ranges_text = f.read()

time: 6.62 ms


In [4]:
occupation_category_map = {}
for category_string in occupation_ranges_text.split("\n"):
    category_string_split = category_string.split(" = ")
    category = category_string_split[0]
    category_range_split = category_string_split[1].split("-")
    start_range = int(category_range_split[0])
    try:
        end_range = int(category_range_split[1])
    except:
        end_range = start_range
    category_range = (start_range, end_range)
    occupation_category_map[category] = category_range

time: 4.44 ms


In [5]:
with open("variable_codes/occupation_range.json", "w") as f:
    json.dump(occupation_category_map, f)

time: 2.83 ms


In [ ]:
def get_occupation(row):
    for category,cat_range in occupation_category_map.items():
        if row["OCC2010"] in range(cat_range[0], cat_range[1]+1):
            return cat_range[0]
    return None

def get_occupation_spouse(row):
    for category,cat_range in occupation_category_map.items():
        if row["OCC2010_SP"] in range(cat_range[0], cat_range[1]+1):
            return cat_range[0]
    return None

In [ ]:
df["OCCUPATION_CATEGORY"] = df.parallel_apply(get_occupation, axis=1)
df["OCCUPATION_CATEGORY_SP"] = df.parallel_apply(get_occupation_spouse, axis=1)

In [ ]:
df.to_csv("usa_small.csv", index=False)